# ***Muziek classificeren***

|Teamleden|Kaggle Username|GitHub Username|
|--|--|--|
|Busse Heemskerk|bussejheemskerk|BJHeemskerk|
|Declan van den Hoek|declanvdh|DeclanvandenHoek|
|Isa Dijkstra|isadijkstra|IsaD01|

In dit notebook gaan we kleine muziek samples classificeren met behulp van unsupervised learning. Een deel van deze bestand heeft een genre label, terwijl de meeste dit niet zullen hebben. Aan ons is de taak om zo accuraat mogelijk te bepalen welke genres de unlabeled samples hebben, door middel van Unsupervised Learning.

Voor het project hebben we gewerkt in [GitHub](https://github.com/BJHeemskerk/MachineLearning/tree/main/Muziek), om makkelijk de bestanden te delen. Van elk model zijn de voorspellingen ook geupload naar [Kaggle](https://www.kaggle.com/competitions/muziek-genre-clustering/overview).

## **Libaries en data inladen** <a name='h1'></a>

In [1]:
import os
import librosa as lr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Om audio af te kunnen spelen
from IPython.display import Audio

In de onderstaande cel, die gebaseerd is op de code van het Machine Learning notebook uit week 11, wordt de data ingelezen en in een dataframe gezet.

In [20]:
# Load labeled data from CSV
labeled_data = pd.read_csv("labels_new.csv", sep=',')
labeled_data = labeled_data.sort_values('filename')

# Map pakken met de juiste samples
base_dir = "labeled"

# Aanmaken lists voor data
audio_data = []
sample_freqs = []

# Lengte is 30 sec op 16000Hz
lengte = 30 * 16000

# Process each audio file
for file in os.listdir(base_dir):
    if file.endswith(".wav"):
        file_path = os.path.join(base_dir, file)
        data, sfreq = lr.load(file_path, sr=None)

        # Truncate or pad the audio
        if len(data) > lengte:
            # Truncate the data
            data = data[:lengte]
        elif len(data) < lengte:
            # Pad with zeros
            padding = lengte - len(data)
            data = np.pad(data, (0, padding), mode='constant')

        # Append the processed data and label
        audio_data.append(data)
        sample_freqs.append(sfreq)

# Convert to numpy array
audio_data = np.stack(audio_data, axis=0)
sample_freqs = np.array(sample_freqs)

# Create a DataFrame from the processed audio data
audio_df = pd.DataFrame({'filename': os.listdir(base_dir), 'data': audio_data.tolist(), 'Hz': sample_freqs})

# Merge the labeled_data DataFrame with the audio_df DataFrame based on the 'Filename' column
df = labeled_data.merge(
    audio_df, how='left', on='filename'
    )

# Display the merged DataFrame
display(df)


,filename,genre,data,Hz
0,m00002.wav,jazz,"[-0.016357421875, -0.0228271484375, -0.0146789...",22050
1,m00039.wav,reggae,"[-0.09478759765625, -0.15338134765625, -0.1439...",22050
2,m00041.wav,pop,"[0.078033447265625, -0.03765869140625, 0.12664...",22050
3,m00072.wav,disco,"[0.1060791015625, 0.0849609375, 0.062103271484...",22050
4,m00096.wav,disco,"[-0.03607177734375, -0.105682373046875, -0.201...",22050
5,m00102.wav,classical,"[0.007843017578125, 0.00726318359375, 0.005828...",22050
6,m00112.wav,reggae,"[-0.00262451171875, -0.003753662109375, -0.003...",22050
7,m00138.wav,reggae,"[-0.02630615234375, -0.028778076171875, -0.013...",22050
8,m00192.wav,classical,"[0.069671630859375, 0.065948486328125, 0.04238...",22050
9,m00206.wav,hiphop,"[-0.1112060546875, -0.198211669921875, -0.1393...",22050


Nu de data is ingeladen kunnen we met behulp van de Audio functie een bestand afspelen in het notebook.

In [29]:
# Pick a random audio clip
random_index = np.random.choice(len(df), size=1, replace=False).item()

# Access the data, Hz, filename, and genre
clip = np.array(df.at[random_index, 'data'])
sfreq = df.at[random_index, 'Hz']
file = df.at[random_index, 'filename']
genre = df.at[random_index, 'genre']

# Print the name and genre
print(f"File and genre: {file}, {genre}")

# Play the clip
Audio(data=clip, rate=sfreq)


File and genre: m00400.wav, blues


## **Feature Engineering** <a name='h2'></a>